# Emerging Technologies Problems

This notebook contains the solutions to the Emerging Technologies problems 1 - 5. It contains markdown cells explaining the code in code cells.

## Problem 1: Generating Random Boolean Functions

## Randomly returns a constant or balanced function from a set of two contant or 3 XOR balanced functions for all 16 possible boolean input combinations. The implmentation details are hidden for security using a closure.

In [1]:
import random

def random_constant_balanced():
    """Returns a randomly chosen constant or balanced function taking 4 Boolean arguments."""
    
    # Constant function that always return 0 
    # for all 16 possible input combinations regardless of inputs
    constant_0 = lambda a, b, c, d: 0

    # Constant function that always return 1
    # for all 16 possible input combinations regardless of inputs
    constant_1 = lambda a, b, c, d: 1
    

    # XOR Balanced function that returns 1 for 8 inputs if the inputs for a and b are different, 
    # and 0 if they are the same
    balanced_1 = lambda a, b, c, d: a ^ b  
    
    # XOR Balanced function that returns 1 for 8 inputs when the sum of (a,b,c) is a odd number
    # and 0 for 8 inputs when the sum of (a,b,c) is a even number
    balanced_2 = lambda a, b, c, d: a ^ b ^ c  
    
    # XOR Balanced function that returns 1 for 8 inputs when the sum of (a,b,c,d) is a odd number
    # and 0 for 8 inputs when the sum of (a,b,c,d) is a even number
    balanced_3 = lambda a, b, c, d: a ^ b ^ c ^ d  
    
    fns = [constant_0, constant_1, balanced_1, balanced_2, balanced_3]
    return random.choice(fns)

# Closure used here to hide implmentations details of the randomly chosen balanced or constant function. 
# The inner function wrapper can access the hidden function _f, but the caller of the wrapper cannot access _f
def hidden_random_constant_balanced():
    """Returns a function with a hidden implementation."""
    
    # Selects a random function to be the hidden implementation
    _f = random_constant_balanced()

    # Inner function defined to call a hidden function in _f
    # so the function arguments cannot be accessed globally
    def wrapper(a, b, c, d):
        return _f(a, b, c, d)
    
    return wrapper

f_hidden = hidden_random_constant_balanced()

print(f_hidden) 
print(f_hidden(0,1,0,1))

<function hidden_random_constant_balanced.<locals>.wrapper at 0x7b62d61768e0>
1


## Problem 2: Classical Testing for Function Type

## This function returns a constant or balanced string depending on whether the chosen function from random_constant_balanced in question 1 is balanced or constant.

In [ ]:
def determine_constant_balanced(f):
    """Determines whether the given function f is constant or balanced."""

    # Loops through all possible 0 and 1 input combinations from function f in problem 1
    results = [f(a, b, c, d) for a in (0, 1) for b in (0, 1) for c in (0, 1) for d in (0, 1)]

    # returns a "constant" string if the output 0 or 1 is contained in the results of all inputs
    if all(result == 0 for result in results) or all(result == 1 for result in results):
        return "constant"
    
    # else returns a "balanced" string if the list of results contain a count of 0s equal the count of 1s
    elif results.count(0) == results.count(1):
        return "balanced"
    

# Loops through function f and determine_constant_balanced functions 10 times
# for "constant" to appear roughly 40% of the time and 
# "balanced" to appear roughly 60% of the time
for i in range(10):
    # Note: Calls function f and determine_constant_balanced functions 
    new_f = hidden_random_constant_balanced() 
    result = determine_constant_balanced(new_f)
    print(f"Test {i+1}: {result}")

Test 1: balanced
Test 2: constant
Test 3: balanced
Test 4: constant
Test 5: balanced
Test 6: balanced
Test 7: balanced
Test 8: constant
Test 9: balanced
Test 10: balanced


## Problem 3: Quantum Oracles

In [17]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator

# Creates a quantum circuit representing the constant function f(x) = 0.
# The oracle maps |x>|y> -> |x>|y XOR 0> = |x>|y>.
def create_oracle_constant_0():
    # Creates a new quantum circuit with 2 qubits.
    # First qubit = input x, second qubit = target y
    qc = QuantumCircuit(2)

    return qc

# Creates a quantum circuit representing the constant function f(x) = 1.
# The oracle maps |x>|y> -> |x>|y XOR 1>.
def create_oracle_constant_1():
    qc = QuantumCircuit(2)

    # Applies a Pauli-X gate to target y to flip the qubit 
    qc.x(1)

    return qc

def demonstrate_oracle(oracle_func, name):
    print(f"{name} Oracle ---")
    sim = AerSimulator()
    
    # Tests all 4 input combinations: 00, 01, 10, 11
    for x_val in [0, 1]:
        for y_val in [0, 1]:
            # Creates a new circuit with 2 qubits and 2 classical bits 
            # to store measurement results.
            qc = QuantumCircuit(2, 2)
            
            # If input x is equal to 1, apply X gate to qubit 0.
            if x_val == 1:
                qc.x(0)

            # If input y is equal to 1, apply X gate to qubit 1.
            if y_val == 1:
                qc.x(1)
                
            # Combines the initialization circuit with the oracle circuit returned by oracle func
            qc = qc.compose(oracle_func())
            
            # Measures qubit 0 into classical bit 0, and qubit 1 into classical bit 1. 
            qc.measure([0, 1], [0, 1])
            
            # Run the circuit on the simulator
            # 1 shot used because there is no superposition involved
            result = sim.run(qc, shots=1).result()
            counts = result.get_counts()

            # Gets the binary string result from the dictionary keys.
            output = list(counts.keys())[0] 
            
            # Qiskit orders output bits right-to-left (q1, q0)
            # So output[0] is q1 (y_out) and output[1] is q0 (x_out)
            x_out = int(output[1])
            y_out = int(output[0])
            
            print(f"Input: x={x_val}, y={y_val} | Output: x={x_out}, y={y_out}")
    print("\n")

# Constant 0
demonstrate_oracle(create_oracle_constant_0, "Constant 0")

# Constant 1
demonstrate_oracle(create_oracle_constant_1, "Constant 1")

Constant 0 Oracle ---
Input: x=0, y=0 | Output: x=0, y=0
Input: x=0, y=1 | Output: x=0, y=1
Input: x=1, y=0 | Output: x=1, y=0
Input: x=1, y=1 | Output: x=1, y=1


Constant 1 Oracle ---
Input: x=0, y=0 | Output: x=0, y=1
Input: x=0, y=1 | Output: x=0, y=0
Input: x=1, y=0 | Output: x=1, y=1
Input: x=1, y=1 | Output: x=1, y=0




## 

## Problem 4: Deutsch's Algorithm with Qiskit

## 

## Problem 5: Scaling to the Deutsch–Jozsa Algorithm

## 